## Note book where I can open a connection to an LLM and give prompts

In [5]:
# Load Libraries
import ollama
import osmnx as ox

In [2]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

 The sky appears blue because of a process called Rayleigh scattering. As sunlight, which is made up of different wavelengths of light, travels through Earth's atmosphere, it interacts with molecules and particles in the air like nitrogen and oxygen. Blue light has shorter wavelengths and gets scattered more easily than other colors due to its energy level differences. This scattered blue light then reaches our eyes from all directions, giving us the overall impression of a blue sky. At sunrise or sunset, the sky can appear red or orange because those longer wavelengths are not absorbed or scattered as much during these hours when the sun is lower in the sky.
 The sky appears blue because of a process called Rayleigh scattering. As sunlight, which is made up of different wavelengths of light, travels through Earth's atmosphere, it interacts with molecules and particles in the air like nitrogen and oxygen. Blue light has shorter wavelengths and gets scattered more easily than other colo

In [7]:
from ollama import chat

stream = chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

 The sky appears blue due to a process called Rayleigh scattering. When sunlight, which is made up of different colors, travels through the atmosphere, it interacts with molecules and small particles in the air like nitrogen and oxygen. Blue light has shorter wavelengths and gets scattered more because it fits easily into the gaps between these molecules and particles. The blue light scatters in all directions, creating a blue sky for us to see. This is also why the sun looks red or orange during sunrise or sunset: At those times of day, sunlight has to pass through more of Earth's atmosphere, which preferentially scatters shorter wavelengths (blue and green), leaving the longer wavelengths like red, orange, and yellow to reach our eyes.

In [2]:
from ollama import chat

model = 'mistral'

messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
  {
    'role': 'assistant',
    'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight.",
  },
  {
    'role': 'user',
    'content': 'What is the weather in Tokyo?',
  },
  {
    'role': 'assistant',
    'content': 'The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing. The city is known for its high-tech and vibrant culture, with many popular tourist attractions such as the Tokyo Tower, Senso-ji Temple, and the bustling Shibuya district.',
  },
]

#while True:
user_input = input('Chat with history: ')
response = chat(
  model,
  messages=[*messages, {'role': 'user', 'content': user_input}],
)

  # Add the response to the messages to maintain the history
messages += [
  {'role': 'user', 'content': user_input},
  {'role': 'assistant', 'content': response.message.content},
]
print(response.message.content + '\n')

 Sorry, I didn't understand your question. Could you please clarify or rephrase it? I'm here to help! If you have any other questions about travel, weather, or culture in Tokyo, feel free to ask!



In [4]:
[*messages, {'role': 'user', 'content': user_input}]

[{'role': 'user', 'content': 'Why is the sky blue?'},
 {'role': 'assistant',
  'content': "The sky is blue because of the way the Earth's atmosphere scatters sunlight."},
 {'role': 'user', 'content': 'What is the weather in Tokyo?'},
 {'role': 'assistant',
  'content': 'The weather in Tokyo is typically warm and humid during the summer months, with temperatures often exceeding 30°C (86°F). The city experiences a rainy season from June to September, with heavy rainfall and occasional typhoons. Winter is mild, with temperatures rarely dropping below freezing. The city is known for its high-tech and vibrant culture, with many popular tourist attractions such as the Tokyo Tower, Senso-ji Temple, and the bustling Shibuya district.'},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': " Sorry, I didn't understand your question. Could you please clarify or rephrase it? I'm here to help! If you have any other questions about travel, weather, or culture in Tokyo, feel free to

## Get the nodes
We will get nodes from the area with the test centre

In [13]:
G = ox.graph_from_point((51.3829463, -0.2933327), dist=5000, network_type='drive')  # Get the OSMNX network of nodes
## Add start nodes
G.add_node('test_centre', x=51.38413, y=-0.29299)
G.add_node('test_centre_road', x=51.38409, y=-0.29280)


In [12]:
list(G.neighbors(23779844))

[23780702, 23780711, 2578590287]

## Make the prompt
This is the prompt that will be given to the model 

In [ ]:
# Useful functions
# Get the nodes on the road
def get_all_nodes_on_street(G, start_node, street_name):
    """This is a 'flood' fill to get all of the nodes on a street

    Args:
        G (_type_): OSMNX network
        start_node (_type_): a starting node on the street
        street_name (_type_): 
    """
    visited = set() # empty set to store nodes
    stack = [start_node] # list of nodes
    
    while stack: # while the stack list is occupied
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        
        # Check all outward and inward edges
        for neighbour in G.neighbors(node): # 
            for key, data in G.get_edge_data(node, neighbour).items():
        
        


# Get the edges between nodes